<h1> Part 2: Random Coefficient Model </h1>

We assumed that everyone has the same taste for each characteristic. However, it's not the case. Additionally, there may be some features of a good that are observable to the consumer, but not to the econometrician. To allow for these, we introduce a new model for the utility (following BLP (1999)):
$$
\begin{equation}
u_{ijt} = -\alpha \frac{p_{jt}}{y_{it}} + x_{jt}\beta + \sum_{k=1}^K \sigma_k v_{ik} x_{jkt} + \xi_{jt} \epsilon_{ijt}
\end{equation}
$$
where $K$ is the number of (observable) characteristics, $\xi$ is the observable-to-consumer-but-not-to-econometrician component, $v_{ik}$ is a random variable that is distributed standard normal, and $y_{it}$ is the income of individual $i$ at time $t$. 

The first term in the utility function was $\log(y_{it}-p_{jt})$ in BLP (1995). But that term is not defined if the price is higher than the income (and in our case, we have some observations where this is true). To eliminate this case, we used the utility function in BLP (1999). 

Now, define $$\delta_{jt} \equiv x_{jt}\beta + \xi_{jt} $$ and $$\mu_{ijt} \equiv -\alpha\frac{p_{jt}}{y_{it}} + \sum_{k=1}^K \sigma_k v_{ik} x_{jkt} $$
Now, we have the following model:
$$
\begin{equation}
u_{ijt} = \delta_{jt} + \mu_{ijt} + \epsilon_{ijt}
\end{equation}
$$

So, again, if individual $i$ buys good $j$, it means that good $j$ provides him more utility than any other good $k$, including the outside good. Therefore, we can say that the probability of individual $i$ buying $j$ at year $t$ is equal to 

$$ \begin{align}
Pr(i \text{ buys good } j \text{ in year } t) &= Pr(u_{ijt}> u_{ikt}, \forall k= {0,1,\ldots,J_t}, k\neq j) \\
&= Pr(\delta_{jt} + \mu_{ijt} + \epsilon_{ijt} > \delta_{kt} + \mu_{ikt} + \epsilon_{ikt}, \forall k= {0,1,\ldots,J_t}, k\neq j)\\
&= \frac{\exp(\delta_{jt} + \mu_{ijt})}{1 + \sum_{k=1}^{J_t} \exp(\delta_{kt} + \mu_{ikt})}
\end{align} 
$$

Assuming that the individuals are distributed with cdf $T$, we can write the probability of good $j$ being sold in year $t$ as
$$
\begin{align}
Pr(\text{good } j \text{ being sold in year } t) &= \int_i Pr( i \text{ buys good } j \text{ in year } t) dT(i) \\
& = \int_i \frac{\exp(\delta_{jt} + \mu_{ijt})}{1 + \sum_{k=1}^{J_t} \exp(\delta_{kt} + \mu_{ikt})} dT(i) \\
& \approx \frac{1}{ns} \sum_{i=1}^{ns} \frac{\exp(\delta_{jt} + \mu_{ijt})}{1 + \sum_{k=1}^{J_t} \exp(\delta_{kt} + \mu_{ikt})} \\
\end{align}
$$

Here is how we code this function.

In [1]:
def s_comp(expdelta0,expmu):
    nom = expdelta0 @ np.ones((1,ns)) * expmu
    denom = np.dot(A,np.dot(A.T,nom))
    frac = nom/(1+denom)
    s_computed = (1/ns)*(frac.sum(1))
    return(s_computed)

To calculate the probabilities, we need $\delta$'s and $\mu$'s. However, we don't know what $\delta$ is. Why? We don't have the data for $\xi_j$. So, how do we calculate $\delta_{jt}$? We approximate it numerically.

Let's write the following recursion

$$
\begin{equation}
\delta^{n+1} = \delta^{n} + \log(s^{DATA}) - \log(s(\theta)) \; \forall n=0,1,2,\ldots
\end{equation}
$$

where $\theta \equiv (\beta,\alpha,\sigma)$, and $\sigma \equiv (\sigma_0,\ldots,\sigma_k)$. Define $\theta_B \equiv (\alpha,\sigma)$ to be the set of parameters outside of $\delta$ for later use.

Define $s(\theta) \equiv Pr(\text{good } j \text{ being sold in year } t)$. So, $\theta$ is the set of parameters that determines $Pr(\text{good } j \text{ being sold in year } t)$. 

BLP shows that the above equation is a "contraction", i.e. for all $\epsilon>0$, there exists an integer $J_\epsilon$ such that for all $m>J_\epsilon$, $||(\delta^{m+1}-\delta^{m}||<\epsilon$. Therefore, we can use the function above to find the "best" $\delta$ that explains the market shares in the data. 


In [2]:
def contraction(sigma,alpha):
    expdelta0 = deltam
    expdelta0.shape = (nrow,1)
    draws = v * np.kron(np.ones((ns,1)),sigma)
    expmu = (np.exp(np.dot(X,draws.T) - np.dot(price,np.ones((1,ns))) * alpha / y))
    dif = 1
    it = 0
    while(dif > tol and it < 1000):
        rat = s_data/s_comp(expdelta0,expmu)
        rat.shape = (nrow,1)
        expdelta1 = expdelta0 * rat
        dif = (abs(expdelta1/expdelta0-1)).max()
        expdelta0 = expdelta1
        it += 1
    return(np.log(expdelta0))

For computational speed, I transformed the recursion above by taking exponentials of both sides. What next?

## GMM estimation

The moment condition we use for the estimation is
$$
\begin{equation}
E[\xi\mid Z] = 0
\end{equation}
$$
where $Z$ is the instrument matrix. Before going into the GMM objective function, let's first calculate $\xi$'s.

We noted earlier that $ \delta_{jt} \equiv x_{jt}\beta + \xi_{jt}$. Now that we "estimated" $\delta_{jt}$, we can back out $\xi_{jt}$. But first, we need $\beta$. 

We can get $\beta$'s by using 2SLS. How? Use the above equation to regress the $\delta$ we got from the contraction on the product characteristics $X$ and $\xi$, by using $Z$ as the instrument for $\xi$. 

$$
\begin{equation}
\hat{\beta} = (X^{'} Z \Phi^{-1} Z^{'} X)^{-1} X^{'} Z \Phi^{-1} Z^{'} \delta(\theta_B)
\end{equation}
$$
where $\Phi \equiv Z^{'}Z$ being the weighting matrix.

Then, use $\hat{\beta}$ to back out $\xi$:
$$ \xi_{jt} = \delta_{jt}(\theta) - x_{jt}\beta $$

Next, define $$g \equiv \frac{1}{J} Z^{'}\xi$$

Finally, define the GMM objective function as

$$ Q(\theta_B) \equiv g^{'}Wg $$

Any symmetric and positive-definite $W$ gives us the consistent estimates for the parameters. So we use the identity matrix, i.e. $W = I$



In [3]:
def obj(params):
    sigma = params[0:5]
    alpha = params[5]
    delta = contraction(sigma,alpha)
    beta = np.dot(prem,delta)
    deltam = delta
    xi = delta - np.dot(X,beta)
    g = (1/nrow)* (np.dot(Z.T,xi))
    ob = np.dot(g.T,g)
    ob = ob[0,0]
    return(ob)

Finally, we find the parameters $\sigma$ and $\alpha$ by solving

$$
\begin{equation}
\min_{\theta_B = (\sigma,\alpha)} Q(\theta_B)
\end{equation}
$$


## Implementation

Step 0: Load necessary packages.

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from numba import jit
import time
from numpy.linalg import inv
from scipy.optimize import minimize

Step 1: Import the data (prepared earlier with the R code above).

In [5]:
data = pd.read_csv("data.csv")

Step 2: Preparation.

In [6]:
ns = 1500
nrow = data.shape[0]

X = data[["const", "hpwt", "air", "mpd", "size"]]
X = np.asarray(X)
sizeX = X.shape[1]
Z = data[["const", "hpwt", "air", "mpd", "size", "own_const", "own_hpwt", "own_air", "own_mpd", "own_size", "all_const", "all_hpwt", "all_air", "all_mpd", "all_size"]]
Z = np.asarray(Z)
phi = np.dot(Z.T,Z)
prem = inv(X.T @ Z @ inv(phi) @ Z.T @ X) @ X.T @ Z @ inv(phi) @ Z.T


A = pd.get_dummies(data["year"])
A = np.asarray(A, dtype = np.float32)
AAT =  np.dot(A,A.T)
nyears = A.shape[1]


np.random.seed(6128)
v = np.random.randn(ns,sizeX)



ym = np.array([2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745, 2.09805, 2.10404, 2.07208, 2.06019, 2.06561, 2.07672, 2.10437, 2.12608, 2.16426, 2.18071, 2.18856, 2.21250, 2.18377])
sigma_y = 1.72
ym.shape = (nyears,1)
np.random.seed(6128)
ar = np.random.randn(nyears,ns)
y = A @ np.exp(ym + sigma_y*ar)


global deltam 
deltam = np.exp(data["logdif"])
deltam = np.asarray(deltam)


s_data = np.asarray(data["share"])

price = np.asarray(data["price"])
price.shape = (nrow,1)


tol = 1e-8



Step 3: Estimation via minimization of GMM objective function with respect to the parameters $\sigma$ and $\alpha$.

In [7]:
init = np.array([3.612, 4.628, 1.818, 1.050, 2.056, 43.501])
t0 = time.time()
soln = minimize(obj,init,method="L-BFGS-B", tol = 1e-2, options={'disp': True},)
elapsed_time = time.time()-t0

In [8]:
print("Time elapsed:",elapsed_time)
print("GMM objective:", soln.fun)
print("sigma: ", abs(soln.x[0:sizeX]),"\n", "alpha: ", soln.x[sizeX],sep="")
print("beta:", (prem @ contraction(soln.x[0:sizeX],soln.x[sizeX])).T[0])

Time elapsed: 1600.7139041423798
GMM objective: 36.4887900203
sigma: [ 0.15931316  5.46924628  5.5880774   0.17531137  5.26241119]
alpha: 48.0007083577
beta: [-5.34348841  5.72331373 -2.25275835  0.23052835  1.43701791]
